# Logging data and predictions

Let's assume we have already run our model on some images and have the predictions handy.
Our inputs have some metadata like latitude, longitude, speed, and temperature.
Our inputs also include the address of our images in S3. Images loaded to Gantry must be
coming from an S3 or GCS bucket. Including the S3 URI tells Gantry where to find the
image.

In [ ]:
import gantry
import gantry.query as gquery
from dotenv import dotenv_values
import os
from pathlib import Path
import pandas as pd

config = dotenv_values(".env")
gantry.init(api_key=config.get("GANTRY_API_KEY"))
GANTRY_APPLICATION = config.get("GANTRY_APP_NAME")

In [ ]:
inputs = pd.read_csv( 
     "https://gantry-demo-data.s3.us-west-2.amazonaws.com/distracted-driving-detection/data/inputs.csv", 
     engine="c", 
     lineterminator="\n", 
)
inputs

Our outputs contain the label output by the model.

In [ ]:
outputs = pd.read_csv( 
     "https://gantry-demo-data.s3.us-west-2.amazonaws.com/distracted-driving-detection/data/outputs.csv", 
     engine="c", 
     lineterminator="\n", 
)
outputs

We also will specify the `join_keys` as the image address in S3. By doing this instead of letting Gantry auto assign a random join_key, we're able to more easily match the feedback with the image it corresponds to.

In [ ]:
join_keys = pd.read_csv( 
     "https://gantry-demo-data.s3.us-west-2.amazonaws.com/distracted-driving-detection/data/join_keys.csv", 
     engine="c", 
     lineterminator="\n", 
)
join_keys = join_keys['join_keys'].values.tolist()
join_keys[0]

The following code logs all this information to our Gantry application. Note that we'll also give Gantry some metadata like the model version. This will allow us to compare the data across versions if we were to improve and redeploy our model.

In [ ]:
gantry.log_records(GANTRY_APPLICATION, inputs=inputs,outputs=outputs,join_keys=join_keys,as_batch=True, tags={"model_version": "v0"})

Say we spent some time going through the images and labeling them all individually. We can then send that feedback to Gantry as well, with the same `join_keys` to let Gantry know how to match the feedback with the original prediction.

In [ ]:
feedback = pd.read_csv( 
     "https://gantry-demo-data.s3.us-west-2.amazonaws.com/distracted-driving-detection/data/feedback.csv", 
     engine="c", 
     lineterminator="\n", 
)
feedback

In [ ]:
gantry.log_records(GANTRY_APPLICATION, feedbacks=feedback, join_keys=join_keys,as_batch=True)